In [0]:
#https://drive.google.com/open?id=1NVfn_L9YoGJloB_rzKh06YCqQNe6DfJj
%cd /content
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

/content


In [0]:
# https://drive.google.com/open?id=1iLrNPNQ3GMZquyE7QhB7l9Yt4mS46r_p
downloaded = drive.CreateFile({'id': '1iLrNPNQ3GMZquyE7QhB7l9Yt4mS46r_p'})
downloaded.GetContentFile('coco_vgg_id_map.txt')

# https://drive.google.com/open?id=1qmDX_URx9rKqjBZczX0hINk5uA4kI9SA
downloaded = drive.CreateFile({'id': '1qmDX_URx9rKqjBZczX0hINk5uA4kI9SA'})
downloaded.GetContentFile('Training Data QA.pickle')

# https://drive.google.com/open?id=1vBVqBh-caYYat7ZXBleJWHGV0al1I4nJ
downloaded = drive.CreateFile({'id': '1vBVqBh-caYYat7ZXBleJWHGV0al1I4nJ'})
downloaded.GetContentFile('Validation Data QA.pickle')


In [0]:
# https://drive.google.com/open?id=1dp5cFJGjCwzND2SXn22N5ON8dnuS2NT5
downloaded = drive.CreateFile({'id': '1dp5cFJGjCwzND2SXn22N5ON8dnuS2NT5'})
downloaded.GetContentFile('model_2_lstm_embed.h5')

# https://drive.google.com/open?id=1OczoYYqeYjRGF8LOYu1wzitHzs40t-ox
downloaded = drive.CreateFile({'id': '1OczoYYqeYjRGF8LOYu1wzitHzs40t-ox'})
downloaded.GetContentFile('model_2_lstm_embed_tok.h5')

In [0]:
!wget http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
!unzip coco.zip

--2019-06-26 20:16:25--  http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip [following]
--2019-06-26 20:16:25--  https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764830639 (729M) [application/zip]
Saving to: ‘coco.zip’

coco.zip            100%[===================>] 729.40M  15.9MB/s    in 50s     

2019-06-26 20:17:16 (14.6 MB/s) - ‘coco.zip’ saved [764830639/764830639]

Archive:  coco.zip
  inflating: coco/dataset.json       
  inflating: coco/readme.txt         
  inflating: coco/vgg_feats.mat      


In [0]:
import scipy.io
mat = scipy.io.loadmat('/content/coco/vgg_feats.mat')

In [0]:
mat["feats"].shape

(4096, 123287)

vgg_feature matrix are the features of images which have been fed into vgg16 architecture, with 16 layers of convolutional neural network and Dense layer(with 4096 units), except the last dense layer(with 1000 units). So each of the image has beeen fed into the model and which gave us the feature vector of each image, in the shape of 4096 X 1. Concatenating each feature vector of every image, give us the matrix of shape 4096 X 123287.
SInce, 82783 + 40504 = 1232287 (Training and valiation data).
  
From coco_vgg_id_map.txt file, we have image_id in order of the columns of vgg_feats matrix.
Selecting top 10 image id from coco_vgg_id_map file, means selecting top 10 columns from vgg_feats matix. 

In [0]:
import sys
import scipy as sc
import scipy.io
import pandas as pd
import numpy as np

def get_coco_features(split, types ):
  %cd /content
    if split == 'train':
        data_path = 'Training Data QA.pickle'
        if ( types == "small"):
            num_data = 10000
        elif (types == "full"):
            num_data = 82783

    elif split == 'val':
        data_path = 'Validation Data QA.pickle'
        if (types == "small"):
            num_data = 2000
        elif (types == "full"):
            num_data = 40504
    else:
        print('Invalid split!')
        sys.exit()
  
    id_map_path = 'coco_vgg_id_map.txt'
    features_path = '/content/coco/vgg_feats.mat'
    img_labels = pd.read_pickle(data_path)[['image_id']].drop_duplicates().values.tolist()
    img_ids = open(id_map_path).read().splitlines()
    features_struct = sc.io.loadmat(features_path)

    id_map = {}
    for ids in img_ids:
        ids_split = ids.split()
        id_map[int(ids_split[0])] = int(ids_split[1])

    VGGfeatures = features_struct['feats']
    nb_dimensions = VGGfeatures.shape[0]
    nb_images = len(img_labels)
    image_matrix = np.zeros((nb_images,nb_dimensions))

    for i in range(nb_images):
        image_matrix[i,:] = VGGfeatures[:,id_map[img_labels[i][0]]]  
    image_matrix.astype('float32')
    return image_matrix[0:num_data]


In [0]:
import numpy as np
import h5py
print('Loading image features ...')
small_img_features_train = get_coco_features('train', types = "small")
small_img_features_val = get_coco_features('val', types = "small")

#combined_image = np.concatenate((img_features_train, img_features_val), axis = 0) #(123287, 4096)

Loading image features ...
/content
/content


In [0]:
import h5py
h5_feats = h5py.File('small_img_features_train.h5', 'w')
h5_feats.create_dataset('small_img_features_train', data = small_img_features_train)
h5_feats.close()

h5_feats_val = h5py.File('small_img_features_val.h5', 'w')
h5_feats_val.create_dataset('small_img_features_val', data = small_img_features_val)
h5_feats_val.close()

In [0]:
print(small_img_features_train.shape)
print(small_img_features_val.shape)

(10000, 4096)
(2000, 4096)


In [0]:
uploaded = drive.CreateFile({'title': 'small_img_features_train.h5'})
uploaded.SetContentFile('small_img_features_train.h5')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'small_img_features_val.h5'})
uploaded.SetContentFile('small_img_features_val.h5')
uploaded.Upload()

In [0]:
print('Loading image features ...')
full_img_features_train = get_coco_features('train', types = "full")
full_img_features_val = get_coco_features('val', types = "full")

import h5py
h5_feats = h5py.File('full_img_features_train.h5', 'w')
h5_feats.create_dataset('full_img_features_train', data = full_img_features_train)
h5_feats.close()

h5_feats_val = h5py.File('full_img_features_val.h5', 'w')
h5_feats_val.create_dataset('full_img_features_val', data = full_img_features_val)
h5_feats_val.close()

print(full_img_features_train.shape)
print(full_img_features_val.shape)

uploaded = drive.CreateFile({'title': 'full_img_features_train.h5'})
uploaded.SetContentFile('full_img_features_train.h5')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'full_img_features_val.h5'})
uploaded.SetContentFile('full_img_features_val.h5')
uploaded.Upload()

Loading image features ...
/content
/content
(82783, 4096)
(40504, 4096)


Now, Lets process the questions.

In [0]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from scipy import io
import operator
import sys
import scipy as sc
from collections import defaultdict
from nltk import word_tokenize
import pandas as pd
import numpy as np
import pickle
import h5py
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


In [0]:
def get_question_tokenizer(types):
    data_path = "Training Data QA.pickle"
    data_path_val = "Validation Data QA.pickle"

    if ( types == "small"):
        num_data = 10000
        num_data_val = 2000
    elif (types == "full"):
        num_data = 248349
        num_data_val = 121512

    df = pd.read_pickle(data_path) 
    df_val = pd.read_pickle(data_path_val)
    questions = df['questions'].values.tolist()
    questions_val = df_val['questions'].values.tolist()
   
    all_question = questions + questions_val
  
    tokenizer = Tokenizer(num_words = 10000)
    tokenizer.fit_on_texts(all_question)

    word_index = tokenizer.word_index

    # Save the tokenizer, so that we can use this tokenizer whenever we need to predict any reviews.
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    #tokenising train data
    train_question_tokenized = tokenizer.texts_to_sequences(questions)      
    questions = pad_sequences(train_question_tokenized, maxlen = 25)          # len(X_train) x 25

    #tokenising validation data
    val_question_tokenized = tokenizer.texts_to_sequences(questions_val)
    questions_val = pad_sequences(val_question_tokenized, maxlen = 25)               # len(X_val) X 25 

    return questions[0:num_data], questions_val[0: num_data_val], word_index
  
def get_questions_matrix(split):
  
    if split == 'train':
        data_path = 'data_train_qa.pickle'
    elif split == 'val':
        data_path = 'data_val_qa.pickle'
    else:
        print('Invalid split!')
        sys.exit()

    df = pd.read_pickle(data_path)
    questions = df[['questions']].values.tolist()
    word_idx = load_idx()
    seq_list = []

    for question in questions:
        words = word_tokenize(question[0])
        seq = []
        for word in words:
            seq.append(word_idx.get(word,0))
        seq_list.append(seq)
    question_matrix = pad_sequences(seq_list)
  
    question_matrix.astype('int32')
    return question_matrix

In [0]:
# create tokenized question data
import h5py
small_question_train_tokenize, small_question_val_tokenize, word_idx = get_question_tokenizer(types = "small")

#combined_question_tokenize = np.concatenate((question_train_tokenize,  question_val_tokenize),axis = 0)



In [0]:
h5f = h5py.File('small_question_train_tokenize.h5', 'w')
h5f.create_dataset('small_question_train_tokenize', data=small_question_train_tokenize)
h5f.close()

h5f_val = h5py.File('small_question_val_tokenize.h5', 'w')
h5f_val.create_dataset('small_question_val_tokenize', data=small_question_val_tokenize)
h5f_val.close()

uploaded = drive.CreateFile({'title': 'small_question_train_tokenize.h5'})
uploaded.SetContentFile('small_question_train_tokenize.h5')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'small_question_val_tokenize.h5'})
uploaded.SetContentFile('small_question_val_tokenize.h5')
uploaded.Upload()

In [0]:
import pickle

file = open("word_idx.pickle", "wb")
pickle.dump(word_idx, file)
file.close()

uploaded = drive.CreateFile({'title': 'word_idx.pickle'})
uploaded.SetContentFile('word_idx.pickle')
uploaded.Upload()

In [0]:
# create tokenized question data
import h5py
full_question_train_tokenize, full_question_val_tokenize, word_idx_full = get_question_tokenizer(types = "full")

h5f = h5py.File('full_question_train_tokenize.h5', 'w')
h5f.create_dataset('full_question_train_tokenize', data=full_question_train_tokenize)
h5f.close()

h5f_val = h5py.File('full_question_val_tokenize.h5', 'w')
h5f_val.create_dataset('full_question_val_tokenize', data=full_question_val_tokenize)
h5f_val.close()

uploaded = drive.CreateFile({'title': 'full_question_train_tokenize.h5'})
uploaded.SetContentFile('full_question_train_tokenize.h5')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'full_question_val_tokenize.h5'})
uploaded.SetContentFile('full_question_val_tokenize.h5')
uploaded.Upload()

import pickle

file = open("word_idx_full.pickle", "wb")
pickle.dump(word_idx_full, file)
file.close()

uploaded = drive.CreateFile({'title': 'word_idx_full.pickle'})
uploaded.SetContentFile('word_idx_full.pickle')
uploaded.Upload()


In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

--2019-06-18 15:02:42--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-06-18 15:02:42--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2019-06-18 15:02:42--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [0]:
import numpy as np
def loadGloveModel(gloveFile, word_index):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    embedding_index = {}
    print("Opened!")
    for j, line in enumerate(f):
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        embedding_index[word] = embedding
    
    print("Done.",len(embedding_index)," words loaded!")
  
    # Now, we need to create embedding matrix.
    EMBEDDING_DIM = 300
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    print(embedding_matrix.shape)
  
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [0]:
import h5py
import pickle
gloveFile = 'glove.840B.300d.txt'
file = open("word_idx.pickle", "rb")
word_idx = pickle.load(file)
file.close()
embedding_matrix_tokenize = loadGloveModel(gloveFile, word_idx) # (16110, 300)


Loading Glove Model
Opened!
Done. 2196016  words loaded!
(16110, 300)


In [0]:
import pickle
file = open("embedding_matrix_tokenize.pickle", "wb")
pickle.dump(embedding_matrix_tokenize, file)
file.close()

uploaded = drive.CreateFile({'title': 'embedding_matrix_tokenize.pickle'})
uploaded.SetContentFile('embedding_matrix_tokenize.pickle')
uploaded.Upload()

In [0]:
import h5py
h5_feats = h5py.File('embedding_matrix_tokenize.h5', 'w')
h5_feats.create_dataset('embedding_matrix_tokenize', data = embedding_matrix_tokenize)
h5_feats.close()

uploaded = drive.CreateFile({'title': 'embedding_matrix_tokenize.h5'})
uploaded.SetContentFile('embedding_matrix_tokenize.h5')
uploaded.Upload()

In [0]:
uploaded = drive.CreateFile({'title': 'tokenizer.pickle'})
uploaded.SetContentFile('tokenizer.pickle')
uploaded.Upload()

In [0]:
uploaded = drive.CreateFile({'title': 'tokenizer_full.pickle'})
uploaded.SetContentFile('tokenizer.pickle')
uploaded.Upload()

In [0]:
questions = ["what is the stripe on the train"]
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


train_question_tokenized = tokenizer.texts_to_sequences(questions)      
questions = pad_sequences(train_question_tokenized, maxlen = 25)          # len(X_train) x 25
questions

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   3,   2,   1, 932,   7,   1,  44]],
      dtype=int32)

Let's process answers.

In [0]:
def int_to_answers():
    %cd /content
    data_path = 'Training Data QA.pickle'
    df = pd.read_pickle(data_path)
    answers = df['multiple_choice_answer'].values.tolist()
    freq = defaultdict(int)
    for answer in answers:
        freq[answer[0].lower()] += 1
    int_to_answer = sorted(freq.items(),key=operator.itemgetter(1),reverse=True)[0:1000]
    int_to_answer = [answer[0] for answer in int_to_answer]
    return int_to_answer

top_answers = int_to_answers()	

def answers_to_onehot():
	top_answers = int_to_answers()
	answer_to_onehot = {}
	for i, word in enumerate(top_answers):
		onehot = np.zeros(1001)
		onehot[i] = 1.0
		answer_to_onehot[word] = onehot
	return answer_to_onehot
	
answer_to_onehot_dict = answers_to_onehot()

def get_answers_matrix(split, types):
  
    if split == 'train':
        data_path = 'Training Data QA.pickle'
        if ( types == "small"):
            num_data = 30000
        elif (types == "full"):
            num_data = 2483490 

    elif split == 'val':
        data_path = 'Validation Data QA.pickle'
        if (types == "small"):
            num_data = 6000
        elif (types == "full"):
            num_data = 1215120 
    else:
        print('Invalid split!')
        sys.exit()
     
    df = pd.read_pickle(data_path)
    answers = df['multiple_choice_answer'].values.tolist()
    answer_matrix = np.zeros((len(answers),1001))
    default_onehot = np.zeros(1001)
    default_onehot[1000] = 1.0
	
    for i, answer in enumerate(answers):
        answer_matrix[i] = answer_to_onehot_dict.get(answer[0].lower(),default_onehot)
	
    answer_matrix.astype('int32')
    return answer_matrix[0:num_data]

/content
/content


In [0]:
import numpy as np
import h5py

print('Loading answers ...')
small_answers_train = get_answers_matrix('train', types = "small") # float64
small_answers_val = get_answers_matrix('val', types = "small")

h5_ans = h5py.File('small_answers_train.h5', 'w')
h5_ans.create_dataset('small_answers_train', data = small_answers_train)
h5_ans.close()

h5_ans_val = h5py.File('small_answers_val.h5', 'w')
h5_ans_val.create_dataset('small_answers_val', data = small_answers_val)
h5_ans_val.close()

Loading answers ...


In [0]:
print(small_answers_train.shape)
print(small_answers_val.shape)

(30000, 1001)
(6000, 1001)


In [0]:
uploaded = drive.CreateFile({'title': 'small_answers_train.h5'})
uploaded.SetContentFile('small_answers_train.h5')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'small_answers_val.h5'})
uploaded.SetContentFile('small_answers_val.h5')
uploaded.Upload()

With all the data in our drive, let's start training VQA. Continue with Notebook#3